In [166]:
from selenium import webdriver
from textacy.preprocess import preprocess_text
from bs4 import BeautifulSoup as bs

import pandas as pd
import numpy as np
import requests
import re
import time

In [167]:
url = 'https://www.indeed.com/advanced_search?q=title%3A%28data+scientist%29+%24100%2C000+-+%24110%2C000&l=United+States&radius=0&limit=50'

In [168]:
# Start the WebDriver
wd = webdriver.Chrome()

In [169]:
# This section scraps the indeed usa website and store the the job categories, job details and estimated salaries in 
# a final_list
start = time.time()

# list of job categories to search
ttl_list = ['data scientist', 'data analyst', 'research scientist', 'business intelligence']

# list of salary ranges to search
sal_list = []
for i in range(0, 300000, 20000):
    sal = '$'+ str(i) + ' - ' + '$' + str(i + 20000)
    sal_list.append(sal)

# Start looping through the job categories list
final_list = []
for category in ttl_list:
    print 'category : ', category
    cat_holding_list = []
    #Start looping through the salary ranges list
    for ssal in sal_list:
        print 'salary range: ', ssal
        # input job category and salary range into 
        wd.get(url)
        ttl = wd.find_element_by_id('as_ttl')
        ttl.clear()
        ttl.send_keys(category)     
    
        sal = wd.find_element_by_id('salary')
        sal.clear()
        sal.send_keys(ssal)

        # define salary as the average of the salary range. This will serve as target for regression later
        sals = re.findall('[\d]+', ssal)
        salary = np.mean([int(x) for x in sals])

        form = wd.find_element_by_tag_name('form')
        form.submit()

        html_page = wd.page_source
        soup = bs(html_page, 'html.parser')

        # Find urls, if any, for subsequent pages and store them in links
        pagelist = soup.find('div', {'class' : 'pagination'})

        # check if more than 1 page of job listings
        if pagelist:
            urls = pagelist.find_all('a')
            # remove the last link which is 'NEXT' on the website 
            urls = urls[:-1]
            # first front page is default page, no url
            links = ['frontpage']
            for link in urls:
                links.append(link.get('href'))
        else:
            links = ['frontpage']

        sal_holding_list = []
        # Looping through pages for a specific salary range
        for link in links:
            print 'page link : ', link
            
            if link == 'frontpage':
                soup = bs(html_page, 'html.parser')

            else:
                try:
                    wd.get('https://www.indeed.com' + link)
                except:
                    continue
                html_page = wd.page_source
                soup = bs(html_page, 'html.parser')

            # Find the job title details, hiring company, location and job description
            joblist = soup.find('td', {'id' : 'resultsCol'})
            # if no result of search found, continue loop
            try:
                no_result = joblist.find('div', {'class' : 'no_results'})
            except:
                continue
            if no_result:
                print 'no result found : ', no_result.find('a')
                continue
                
            try:
                jobs = joblist.find_all('div', {'class' : 'result'})
            except:
                continue

            job_details = []

            for job in jobs:

                details = job.find('h2', {'class' : 'jobtitle'})
                # find job title  
                try:
                    job_title = details.find('a').get('title').lower()
                    # clean title using textacy.preprocess_text
                    clean_title = preprocess_text(job_title, fix_unicode=True, lowercase=True, transliterate=False, 
                                                no_numbers=True, no_urls=True, no_emails=False, no_phone_numbers=False, 
                                                no_currency_symbols=True, no_punct=True, no_accents=True)
                except:
                    continue

                # find company name
                try:
                    company = job.find('span', {'class' : 'company'}).find('a').text.strip().lower()
                except: 
                    continue

                # find location
                try:
                    location = job.find('span', {'class' : 'location'}).text.strip().lower()
                except:
                    continue

                # find job description
                href = details.find('a').get('href')
                try:
                    wd.get('https://www.indeed.com' + href)
                except:
                    continue
                page = wd.page_source
                soupy = bs(page, 'html.parser')

                try:
                    job_des = soupy.find('body').find_all('p')
                except:
                    continue

                if job_des:

                    descrps = []
                    for des in job_des:
                        if des.text:
                            descrps.append(des.text.strip().lower())
                    job_descrip = ' '.join(descrps)    

                    # clean job description using textacy.preprocess_text
                    clean_descrip = preprocess_text(job_descrip, fix_unicode=False, lowercase=True, transliterate=False, 
                                            no_numbers=True, no_urls=True, no_emails=True, no_phone_numbers=True, 
                                            no_currency_symbols=True, no_punct=True, no_accents=True)
                    # check if content is long enough, else likely just to be some kind of notices, not relevant
                    if len(clean_descrip) > 2000:
                        print 'job description snippet : ', clean_descrip[:50]
                        # create list of all the job details
                        job_details.append([category, clean_title, company, location, clean_descrip, salary])

            sal_holding_list.extend(job_details)

        cat_holding_list.extend(sal_holding_list) 

    final_list.extend(cat_holding_list)
        
end = time.time()
exe_time = (end - start)/3600
print 'Time taken :',(exe_time),' hours'

category :  data scientist
salary range:  $0 - $20000
page link :  frontpage
salary range:  $20000 - $40000
page link :  frontpage
job description snippet :  please use google chrome when applying for job to 
salary range:  $40000 - $60000
page link :  frontpage
job description snippet :  job title production control data scientist depart
job description snippet :  the microsoft knowledge graph team in the artifici
job description snippet :  schneider electrictm creates connected technologie
job description snippet :  the university of chicago is an affirmative action
job description snippet :  cleveland ohio fulltime paid college students part
job description snippet :  numbersepnumber san bruno ca internship position d
job description snippet :  discover the cia history mission vision and values
job description snippet :  the microsoft knowledge graph team in the ai+r cor
job description snippet :  this website uses cookies by navigating around thi
job description snippet :  numberse

job description snippet :  numberaugnumber austin tx software development and
job description snippet :  make tomorrow better that is the essence of roles 
job description snippet :  discover the cia history mission vision and values
job description snippet :  questions about our site please click here the cig
job description snippet :  we created toptal to solve an extremely difficult 
job description snippet :  at quintiles we help healthcare and biopharma comp
job description snippet :  sap is committed to the principles of equal employ
job description snippet :  title data scientist advanced analytics grade numb
job description snippet :  when youre part of the team at thermo fisher scien
job description snippet :  key focus the data scientist will support product 
page link :  /jobs?q=title%3A%28data+scientist%29+%2480%2C000+-+%24100%2C000&l=United+States&radius=0&limit=50&start=50
job description snippet :  110131br job description start and end dates for t
job description snippe

job description snippet :  ucsd layoff from career appointment apply by numbe
job description snippet :  keyword search city state or zip search radius
rad
job description snippet :  date oct number number location new york ny us num
job description snippet :  are you passionate about applying machine learning
job description snippet :  our vision at petco is healthier pets happier peop
job description snippet :  job description the leidos global services group g
job description snippet :  keyword location radius
select radius
number miles
job description snippet :  keyword location radius
select radius
number miles
job description snippet :  senior data scientist if youre looking for a caree
job description snippet :  sap is committed to the principles of equal employ
job description snippet :  the civil health operation of leidos is seeking a 
job description snippet :  untuckit was founded in number based on the idea o
job description snippet :  maybe you would like to consider the 

job description snippet :  pluralsight proudly creates the creators of tomorr
job description snippet :  see the product page for additional features and m
job description snippet :  keyword search city state or zip search radius
rad
job description snippet :  number mountain view california fuel small busines
job description snippet :  apply with linkedin terms of use big data scientis
job description snippet :  dailymotion is the leading video discovery destina
job description snippet :  science chicago il st louis or seattle
 washington
job description snippet :  at mathematica we take pride in our employees and 
job description snippet :  this position is an exciting opportunity in attrib
job description snippet :  ref number cbs business unit showtime job type ful
job description snippet :  the role we are looking for a data scientist for t
page link :  /jobs?q=title%3A%28data+scientist%29+%24100%2C000+-+%24120%2C000&l=United+States&radius=0&limit=50&start=150
job description snip

job description snippet :  comma separated job title or keyword october numbe
job description snippet :  comma separated job title or keyword october numbe
job description snippet :  keyword location radius
select radius
number miles
job description snippet :  comma separated job title or keyword october numbe
job description snippet :  sharp laboratories of america sla is looking for a
job description snippet :  comma separated job title or keyword september num
page link :  /jobs?q=title%3A%28data+scientist%29+%24100%2C000+-+%24120%2C000&l=United+States&radius=0&limit=50&start=450
job description snippet :  are you an experienced manager passionate about ap
job description snippet :  verisk has an amazing story to tellnamed to forbes
job description snippet :  this site uses cookies to provide you with a more 
job description snippet :  selecting third party means you are a recruiter or
job description snippet :  by continuing to use this website you consent to o
job description snip

...

job description snippet :  johnson johnson family of companies are equal oppo
job description snippet :  at fitbit our mission is to help people lead healt
job description snippet :  based in new york and cambridge digitalocean is a 
job description snippet :  reference number number
location midwest our clien
salary range:  $140000 - $160000
page link :  frontpage
job description snippet :  salesforce research previously metamind is looking
job description snippet :  important information for arm careersthis site use
job description snippet :  position description aig is seeking a dynamic indi
salary range:  $160000 - $180000
page link :  frontpage
salary range:  $180000 - $200000
page link :  frontpage
no result found :  <a id="jobPostingsAnchor" tabindex="-1"></a>
salary range:  $200000 - $220000
page link :  frontpage
no result found :  <a id="jobPostingsAnchor" tabindex="-1"></a>
salary range:  $220000 - $240000
page link :  frontpage
no result found :  <a id="jobPostingsAnchor" t

job description snippet :  apply now not ready to apply but want to stay upto
job description snippet :  below you will find the details for the position i
job description snippet :  at panda we all share a common mission deliver exc
job description snippet :  minimum qualifications include bachelors degree in
job description snippet :  site people the university of miami is considered 
job description snippet :  standard university business hours number30am numb
job description snippet :  if you have a disability and need assistance compl
job description snippet :  technical capabilitiesbusiness intelligence tools 
job description snippet :  menu keyword location radius
radius
number miles
n
job description snippet :  why beenverified beenverified builds exciting data
job description snippet :  csu job listing details printable version
email th
job description snippet :  job number r0012090 location san antonio tx date p
job description snippet :  date sep number number location boise

job description snippet :  hot topic hq city of industry california fulltime 
job description snippet :  wealth managementopen an accountnationwide branche
job description snippet :  requisition number
location johns hopkins hospital
job description snippet :  keyword location radius
miles
number miles
number 
job description snippet :  requisition number
location johns hopkins hospital
job description snippet :  do you want to join one of the worlds most admired
job description snippet :  position description who is domtar we design manuf
job description snippet :  microsoft data engineer slalom consulting denver a
job description snippet :  careers mt culture inside mt training programs div
job description snippet :  who we are with decades of experience in the subst
page link :  /jobs?q=title%3A%28business+intelligence%29+%2480%2C000+-+%24100%2C000&l=United+States&radius=0&limit=50&start=150
job description snippet :  what will i be doingas a business intelligence dev
job descriptio

job description snippet :  view all view all view all as part of advocate hea
page link :  /jobs?q=title%3A%28business+intelligence%29+%2480%2C000+-+%24100%2C000&l=United+States&radius=0&limit=50&start=400
job description snippet :  psrp salary range c45 with a starting salary betwe
job description snippet :  why youll love it herewe are abarca health a fastg
job description snippet :  keyword location radius
select
number miles
number
job description snippet :  mastech digital is an it staffing and digital tran
job description snippet :  comma separated job title or keyword september num
job description snippet :  comma separated job title or keyword september num
job description snippet :  avvo is seeking an experienced analyst with a trac
job description snippet :  we have big plans for continued global growth and 
job description snippet :  comma separated job title or keyword september num
job description snippet :  comma separated job title or keyword september num
page link :  /

page link :  /jobs?q=title%3A%28business+intelligence%29+%24100%2C000+-+%24120%2C000&l=United+States&radius=0&limit=50&start=200
job description snippet :  connecting decision makers to a dynamic network of
job description snippet :  work for the select group business intelligence ma
job description snippet :  this website uses cookies to give you the best use
job description snippet :  this site uses cookies to provide you with a more 
job description snippet :  comma separated job title or keyword october numbe
job description snippet :  by continuing to use this website you consent to o
job description snippet :  our client is looking for a sr business intelligen
job description snippet :  by continuing to use this website you consent to o
job description snippet :  hpe websites use data collection tools such as coo
job description snippet :  hpe websites use data collection tools such as coo
job description snippet :  selecting third party means you are a recruiter or
job descripti

In [170]:
print len(final_list)

2541


In [171]:
df = pd.DataFrame(final_list, columns=['job_category', 'job_title', 'company_name', 'location', 'job_description', 'salary'])

In [172]:
len(df)

2541

In [173]:
df.drop_duplicates(inplace=True)

In [174]:
len(df)

2538

In [175]:
df['job_category'].value_counts()

data analyst             1013
data scientist            678
business intelligence     448
research scientist        399
Name: job_category, dtype: int64

In [177]:
df.to_csv('job_results.csv', encoding='utf-8')

In [178]:
wd.quit()